In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
pd.set_option('display.max_columns', 500)
sns.set_style("ticks")

# I - Importation et nettoyage des données

In [4]:
data = pd.read_csv('/home/cesar/Desktop/data/atp_matches_2010.csv')
for annee in range(2000, 2020):
    data = pd.concat([data, pd.read_csv('/home/cesar/Desktop/data/atp_matches_'+str(annee)+'.csv')], axis=0, sort=False)

In [5]:
data['tourney_date'] = pd.to_datetime(data['tourney_date'], yearfirst=True, format='%Y%m%d')
data['year'] = data['tourney_date'].apply(lambda x: x.year)
data = data[~data['tourney_name'].str.startswith('Davis Cup')].copy()
data.dropna(subset=['minutes'], inplace=True)
data = data[~data['score'].str.contains('RET')].copy()

In [6]:
data_winner = data[data.columns]
data_loser = data[data.columns]
data_winner.rename({'winner_name':'player_name'}, inplace=True)
data_loser.rename({'loser_name':'player_name'}, inplace=True)
data_winner['win_lose'] = ['w' for i in range(len(data_winner))]
data_loser['win_lose'] = ['l' for i in range(len(data_loser))]
data_winner = data_winner.rename(columns=lambda x: re.sub('winner','player',x))
data_winner = data_winner.rename(columns=lambda x: re.sub('^w_','player_',x))
data_winner = data_winner.rename(columns=lambda x: re.sub('loser','opponent',x))
data_winner = data_winner.rename(columns=lambda x: re.sub('^l_','opponent_',x))
data_loser = data_loser.rename(columns=lambda x: re.sub('loser','player',x))
data_loser = data_loser.rename(columns=lambda x: re.sub('^l_','player_',x))
data_loser = data_loser.rename(columns=lambda x: re.sub('winner','opponent',x))
data_loser = data_loser.rename(columns=lambda x: re.sub('^w_','opponent_',x))
data = pd.concat([data_winner, data_loser], axis=0, sort=False)
data.reset_index(inplace=True, drop=True)
data.drop_duplicates(inplace=True)

In [7]:
data.head(3)

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,player_id,player_seed,player_entry,player_name,player_hand,player_ht,player_ioc,player_age,opponent_id,opponent_seed,opponent_entry,opponent_name,opponent_hand,opponent_ht,opponent_ioc,opponent_age,score,best_of,round,minutes,player_ace,player_df,player_svpt,player_1stIn,player_1stWon,player_2ndWon,player_SvGms,player_bpSaved,player_bpFaced,opponent_ace,opponent_df,opponent_svpt,opponent_1stIn,opponent_1stWon,opponent_2ndWon,opponent_SvGms,opponent_bpSaved,opponent_bpFaced,player_rank,player_rank_points,opponent_rank,opponent_rank_points,year,win_lose
0,2010-339,Brisbane,Hard,NaN,A,2010-01-03,1,104053,1,NaN,Andy Roddick,R,188.0,USA,27.35,103429,NaN,NaN,Peter Luczak,R,183.0,AUS,30.34,7-6(5) 6-2,3,R32,84.0,15.0,0.0,63.0,42.0,36.0,14.0,10.0,3.0,3.0,4.0,2.0,56.0,34.0,29.0,11.0,10.0,3.0,5.0,7.0,4410.0,77.0,598.0,2010,w
1,2010-339,Brisbane,Hard,NaN,A,2010-01-03,2,104958,NaN,WC,Carsten Ball,L,198.0,AUS,22.54,104999,NaN,NaN,Mischa Zverev,L,190.0,GER,22.37,7-5 6-1,3,R32,70.0,10.0,3.0,57.0,30.0,23.0,19.0,10.0,0.0,0.0,2.0,2.0,66.0,34.0,22.0,14.0,9.0,7.0,10.0,134.0,400.0,78.0,590.0,2010,w
2,2010-339,Brisbane,Hard,NaN,A,2010-01-03,3,104755,NaN,NaN,Richard Gasquet,R,185.0,FRA,23.55,103813,NaN,NaN,Jarkko Nieminen,L,185.0,FIN,28.45,6-3 4-6 6-4,3,R32,121.0,5.0,4.0,97.0,51.0,33.0,27.0,15.0,5.0,8.0,4.0,0.0,85.0,58.0,38.0,14.0,14.0,7.0,11.0,52.0,850.0,88.0,568.0,2010,w


# II - Un premier modèle de base

Nous commençons par construire un modèle qui nous servira de baseline pour juger de la qualité des modèles qui le suivront. La mesure de qualité du modèle choisi sera la Mean Squared Error (MSE).  
  
La plupart de nos variables sont des statitstiques concernant directement le match et ne sont donc pas utilisable lors d'une prédiction. Ces variables demanderont d'être aggrégés par la suite afin d'être utilisable pour caractériser un joueur.  
  
Ce modèle de base s'appuiera sur une régression linéaire, ce qui demande au préalable de transformer les variables catégorielles en dummies.

### 2.1 Mise en forme des données

In [8]:
X = data[['tourney_name', 'surface', 'tourney_level', 'player_hand', 'player_age', 'player_rank', 'opponent_hand', 'opponent_age', 'opponent_rank', 'best_of', 'round', 'year']].copy()
y = data['minutes']

X.shape, y.shape

((104036, 12), (104036,))

On commence par retirer les lignes contenant des valeurs manquantes (non prises en charge par les algorithmes de régressions linéaires)

In [9]:
for col in X.columns:
    print(col, X[col].isna().sum())

tourney_name 0
surface 0
tourney_level 0
player_hand 0
player_age 0
player_rank 150
opponent_hand 0
opponent_age 0
opponent_rank 150
best_of 0
round 0
year 0


In [10]:
X.dropna(axis=0, how='any', inplace=True)
y = y[X.index]

X.shape, y.shape

((103736, 12), (103736,))

### One Hot Encoding des variables catégorielles

Nous observons le nombre de modalitées prises par les variables catégorielles traités

In [11]:
X_qual = ['tourney_name', 'surface', 'tourney_level', 'round', 'player_hand', 'opponent_hand']

In [12]:
for variable in X_qual:
    print(variable, ':', pd.get_dummies(X[variable]).shape[1])

tourney_name : 133
surface : 4
tourney_level : 4
round : 9
player_hand : 3
opponent_hand : 3


La variable `tourney_name` prend 133 modalités. La matrice étant trop sparse nous l'ignorons pour le moment.  
Les autres variables qualitatives seront retraitées de manière à ce que chaque modalité soit encodée par une dummy, en prenant garde au problème de multicolinéarité en utilisant l'option `drop_first=True` de la fonction `get_dummies`.

In [13]:
X_qual.remove('tourney_name')
X.drop(columns='tourney_name', inplace=True)

In [14]:
for variable in X_qual:
    var_enc = pd.get_dummies(X[variable], prefix=variable, drop_first=True)
    X.drop(variable, axis=1, inplace=True)
    for col_enc in var_enc.columns:
        X[col_enc] = var_enc[col_enc]

### 2.2 Régression linéaire de `sklearn`

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Nous utilisons finalement la fonction `LinearRegression` en notant que l'intercept est mise par défaut par la fonction (et la gestion de la multicolinéarité est égaelement gérée par défaut par la fonction) 

In [16]:
reg = LinearRegression().fit(X.values, y.values)

y_pred = reg.predict(X.values)
print('MSE :', mean_squared_error(y.values, y_pred),
      '\nRsquared :', reg.score(X.values, y.values))

MSE : 1146.009308459461 
Rsquared : 0.27316682703886064


### 2.3 Régression linéaire de `statsmodels`

A l'aide de `statsmodels` qui nous donne le Adj. R-squared qui nous sera utile suite à l'ajout de la variable `tourney_name`. Il faut cependant cette fois-ci introduire une constante dans notre modèle avec la fonction `add_constant` car la fonction `ols` n'en utilise pas par défaut.

In [17]:
from statsmodels.formula.api import ols
from statsmodels.tools.tools import add_constant
modele = ols('minutes ~ const + C(tourney_name) + C(surface) + C(tourney_level) + C(player_hand)+ player_age + player_rank\
          + C(opponent_hand) + opponent_age + opponent_rank + best_of + C(round) + year', data=add_constant(data))
resultat = modele.fit()

In [18]:
print('MSE :', resultat.mse_resid,
      '\nRsquared :', resultat.rsquared,
      '\nAdjusted Rsquared :', resultat.rsquared_adj)

MSE : 1142.0776543658171 
Rsquared : 0.2767426909028152 
Adjusted Rsquared : 0.27566738147733216


# III - Création de variables relatives à la rencontre

In [19]:
print('Features de base :')
print(data.columns.values)

Features de base :
['tourney_id' 'tourney_name' 'surface' 'draw_size' 'tourney_level'
 'tourney_date' 'match_num' 'player_id' 'player_seed' 'player_entry'
 'player_name' 'player_hand' 'player_ht' 'player_ioc' 'player_age'
 'opponent_id' 'opponent_seed' 'opponent_entry' 'opponent_name'
 'opponent_hand' 'opponent_ht' 'opponent_ioc' 'opponent_age' 'score'
 'best_of' 'round' 'minutes' 'player_ace' 'player_df' 'player_svpt'
 'player_1stIn' 'player_1stWon' 'player_2ndWon' 'player_SvGms'
 'player_bpSaved' 'player_bpFaced' 'opponent_ace' 'opponent_df'
 'opponent_svpt' 'opponent_1stIn' 'opponent_1stWon' 'opponent_2ndWon'
 'opponent_SvGms' 'opponent_bpSaved' 'opponent_bpFaced' 'player_rank'
 'player_rank_points' 'opponent_rank' 'opponent_rank_points' 'year'
 'win_lose']


### 3.1 Ecart de classement entre les deux joueurs

In [20]:
data['diff_ranking'] = data.apply(lambda x: abs(x.player_rank - x.opponent_rank), axis=1)
data['diff_ranking'].head(3)

0    70.0
1    56.0
2    36.0
Name: diff_ranking, dtype: float64

### 3.2 Classement moyen des deux joueurs du match

In [21]:
data['avg_ranking'] = data.apply(lambda x: (x.player_rank + x.opponent_rank)/2, axis=1)
data['avg_ranking'].head(3)

0     42.0
1    106.0
2     70.0
Name: avg_ranking, dtype: float64

### 3.3 Les joueurs jouent de la même main

In [22]:
data['same_hand'] = data.apply(lambda x: 1 if x['player_hand']!=x['opponent_hand'] else 0, axis=1)
data['same_hand'].head(3)

0    0
1    0
2    1
Name: same_hand, dtype: int64

# IV - Création de variables caractéristiques du joueur 

Dans cette partie nous devons isoler les matchs joués par chacun des joueurs présents dans notre base pour créer des variables qui leur sont personnelles

In [23]:
nb_joueur = data['player_name'].unique().shape[0]
print('Nombre de joueur différents dans notre base :', nb_joueur)

Nombre de joueur différents dans notre base : 1355


### 4.1 Temps de match moyen d'un joueur

In [24]:
mean_time = data.groupby(['player_name'])['minutes'].mean()
data['mean_time'] = np.zeros(data.shape[0])
data['mean_time'] = data.apply(lambda x: mean_time[x['player_name']], axis=1)
data[['player_name', 'mean_time']].head(3)

,player_name,mean_time
0,Andy Roddick,99.924202
1,Carsten Ball,111.421053
2,Richard Gasquet,104.191462


### 4.1 Nombre de sets moyen joué par un joueur

In [25]:
import string

In [36]:
data['nb_set'] = data.apply(lambda x: x['score'].split(' '), axis=1)
data['nb_set'] = data.apply(lambda x: [string.split('-') for string in x['nb_set']], axis=1)

In [37]:
data['nb_set'] = data.apply(lambda x: [[int(elt[0]) if elt[0] not in string.ascii_letters else liste.remove(elt) for elt in liste]
                                         if len(liste)>1 else x['nb_set'].remove(liste) for liste in x['nb_set']], axis=1)

In [43]:
data['nb_set'] = data.apply(lambda x: [x['nb_set'].remove(liste) for liste in x['nb_set'] if liste==None], axis=1)

In [139]:
data['nb_set'] = data.apply(lambda x: [sum(liste) for liste in x['nb_set']], axis=1)

TypeError: ("'NoneType' object is not iterable", 'occurred at index 3932')

In [44]:
data.iloc[3932]

tourney_id                         2000-520
tourney_name                  Roland Garros
surface                                Clay
draw_size                               NaN
tourney_level                             G
tourney_date            2000-05-29 00:00:00
match_num                                70
player_id                            102722
player_seed                             NaN
player_entry                              Q
player_name                   Attila Savolt
player_hand                               R
player_ht                               183
player_ioc                              HUN
player_age                            24.31
opponent_id                          102905
opponent_seed                           NaN
opponent_entry                          NaN
opponent_name                 Stefan Koubek
opponent_hand                             L
opponent_ht                             175
opponent_ioc                            AUT
opponent_age                    

In [42]:
a = [[3, 6], [7, 5], [6, 0], [5, 2], None]
[a.remove(liste) for liste in a if liste==None]
a

[[3, 6], [7, 5], [6, 0], [5, 2]]

### 4.2 Nombre de sets moyen d'un joueur moyenne sur ses 10 derniers matchs